In [1]:
import json
from pathlib import Path
import shutil
from typing import List, Tuple


def get_images_with_something(root_dir: Path, min_score: float = 0.00):
    with_something = []
    n_inferences = 0
    inference_dir = root_dir / "inferences"
    img_dir = root_dir / "visuals_with_gt"
    for fpath in sorted(inference_dir.rglob("*.json")):
        n_inferences += 1
        try:
            with open(fpath) as f:
                detections = json.load(f)
        except json.JSONDecodeError:
            raise RuntimeError(f"Error decoding file {fpath}")
        detections_with_something = [i for i in detections if i["score"] >= min_score]
        if detections_with_something:
            img_path = img_dir / fpath.parent.relative_to(inference_dir) / f"{fpath.stem}.jpg"
            if not img_path.exists():
                print("No image!", img_path)
                continue
            with_something.append((img_path, img_path.relative_to(img_dir)))
    print(f"Found {len(with_something)} inferences from {n_inferences} with something")
    return with_something


def copy_to_dir(odir: Path, with_something: List[Tuple[Path, Path]]):
    for imgpath, relpath in with_something:
        opath = odir / relpath.parent / f"{imgpath.name}"
        if not opath.exists():
            opath.parent.mkdir(exist_ok=True, parents=True)
            shutil.copy(imgpath, opath)


def process_dir(root_dir: Path, min_score: float = 0.00):
    with_something = get_images_with_something(root_dir, min_score)
    if not with_something:
        print("No images found")
        return
    odir = root_dir / "to-review-ie-with-something"
    odir.mkdir(exist_ok=True, parents=True)
    copy_to_dir(odir, with_something)


for res_dir in sorted(
    Path("/workspaces/cv/data/inferences/20240815_060150-yolov9-c-20240815-maui/640x640_0.2x0.2/0.01_0.5").iterdir()
):
    if res_dir.is_dir():
        if (res_dir / "reviewed").exists():
            continue
        print(f"Processing {res_dir}")
        process_dir(res_dir)

Processing /workspaces/cv/data/inferences/20240815_060150-yolov9-c-20240815-maui/640x640_0.2x0.2/0.01_0.5/20240822-r09
Found 211 inferences from 7560 with something
